In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27733850
paper_name = 'gonzalez_valero_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [28]:
sheet_names = ['Sorbitol vs. Basal','Glucose vs. Basal']
original_data_list = []

In [29]:
for s in sheet_names:
    original_data = pd.read_excel('raw_data/Data_Sheet_1.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data['orf'] = original_data.iloc[:,0].astype(str)
    original_data['orf'] = original_data['orf'].apply(lambda x: x.split('_')[0])
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    
    # Make sure everything translated ok
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = original_data['logFC']
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 4300 x 3
Empty DataFrame
Columns: [YS, logFC, FDR, orf]
Index: []
(4119, 1)
Original data dimensions: 4300 x 3
Empty DataFrame
Columns: [YG, logFC, FDR, orf]
Index: []
(4119, 1)


In [30]:
original_data1, original_data2 = original_data_list

In [31]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [32]:
original_data.head()

,data_1,data_2
orf,,
YAL002W,-0.300982,1.012886
YAL004W,-0.159044,0.081111
YAL005C,-0.302876,0.240369
YAL007C,0.083283,0.599999
YAL008W,0.003321,0.197171


# Prepare the final dataset

In [33]:
data = original_data.copy()

In [34]:
dataset_ids = [5183, 5184]
datasets = datasets.reindex(index=dataset_ids)

In [35]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [36]:
data.head()

dataset_id,5183,5184
data_type,value,value
orf,,
YAL002W,-0.300982,1.012886
YAL004W,-0.159044,0.081111
YAL005C,-0.302876,0.240369
YAL007C,0.083283,0.599999
YAL008W,0.003321,0.197171


## Subset to the genes currently in SGD

In [37]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [38]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [39]:
data.head()

,dataset_id,5183,5184
,data_type,value,value
gene_id,orf,,
2,YAL002W,-0.300982,1.012886
1863,YAL004W,-0.159044,0.081111
4,YAL005C,-0.302876,0.240369
5,YAL007C,0.083283,0.599999
6,YAL008W,0.003321,0.197171


# Normalize

In [40]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [41]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [42]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [43]:
data_all.head()


,dataset_id,5183,5184,5183,5184
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,-0.300982,1.012886,-0.037082,0.281941
1863,YAL004W,-0.159044,0.081111,-0.009291,0.079363
4,YAL005C,-0.302876,0.240369,-0.037453,0.113988
5,YAL007C,0.083283,0.599999,0.038156,0.192175
6,YAL008W,0.003321,0.197171,0.022499,0.104596


# Print out

In [44]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
from IO.save_data_to_db3 import *

In [46]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 27733850...
Inserting the new data...


100%|██████████| 2/2 [00:13<00:00,  6.61s/it]

Updating the data_modified_on field...
